In [1]:
import pandas as pd
import datetime as dt

In [2]:
start=dt.datetime(2021,6,1)
end=dt.datetime(2022,2,11)
stk_data=pd.read_csv("Tatacoffee13_21.csv")

In [3]:
stk_data

,Date,Open,High,Low,Close
0,2013-01-01,1410.60,1427.90,1408.30,1415.10
1,2013-01-02,1421.00,1626.60,1416.15,1607.40
2,2013-01-03,1632.55,1673.90,1613.05,1626.20
3,2013-01-04,1627.75,1627.75,1574.60,1579.05
4,2013-01-07,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...,...
2220,2021-12-22,202.90,207.80,201.35,205.00
2221,2021-12-23,206.00,206.85,202.05,202.95
2222,2021-12-24,203.90,203.90,199.35,201.00
2223,2021-12-27,200.00,222.00,196.00,218.35


In [4]:
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2225 non-null   object 
 1   Open    2225 non-null   float64
 2   High    2225 non-null   float64
 3   Low     2225 non-null   float64
 4   Close   2225 non-null   float64
dtypes: float64(4), object(1)
memory usage: 87.0+ KB


In [5]:
stk_data['Date']=pd.to_datetime(stk_data['Date'])

In [6]:
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2225 non-null   datetime64[ns]
 1   Open    2225 non-null   float64       
 2   High    2225 non-null   float64       
 3   Low     2225 non-null   float64       
 4   Close   2225 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 87.0 KB


In [7]:
stk_data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close'], dtype='object')

In [8]:
startdate="2021-6-1" 
enddate="2022-2-11"    
df=stk_data[stk_data['Date'].between(startdate,enddate)][['Open', 'High', 'Low', 'Close']]

In [9]:
df

,Open,High,Low,Close
2080,180.00,181.35,172.65,174.25
2081,173.55,175.65,172.05,174.00
2082,176.90,178.70,175.60,176.70
2083,177.90,177.90,173.75,174.35
2084,176.40,176.65,173.00,174.35
...,...,...,...,...
2220,202.90,207.80,201.35,205.00
2221,206.00,206.85,202.05,202.95
2222,203.90,203.90,199.35,201.00
2223,200.00,222.00,196.00,218.35


In [10]:
#column='Open', 'High', 'Low', 'Close'

In [11]:
#column

In [12]:
df=df[["Open","High","Low","Close"]]

In [13]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1=Ms.fit_transform(df)
print("Len:",data1.shape)

Len: (145, 4)


In [14]:
len(data1)

145

In [15]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [16]:
data1

,Open,High,Low,Close
0,0.140240,0.072381,0.073518,0.021463
1,0.049331,0.000000,0.064032,0.017488
2,0.096547,0.038730,0.120158,0.060413
3,0.110641,0.028571,0.090909,0.023052
4,0.089500,0.012698,0.079051,0.023052
...,...,...,...,...
140,0.463002,0.408254,0.527273,0.510334
141,0.506695,0.396190,0.538340,0.477742
142,0.477097,0.358730,0.495652,0.446741
143,0.422128,0.588571,0.442688,0.722576


In [17]:
training_size=round(len(data1)*0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_trian length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

116
X_trian length: (116, 4)
X_test length: (29, 4)
y_train length: (116, 4)
y_test length: (29, 4)


In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [20]:
performance

{'Model': [], 'RMSE': [], 'MaPe': [], 'Lag': [], 'Test': []}

In [21]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [22]:
listt=["Close","High","Open","Low"]

In [23]:
listt

['Close', 'High', 'Open', 'Low']

In [24]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -26.345526237461907
BIC:  -25.87076930796081

Order = 2
AIC:  -26.148722654741817
BIC:  -25.289439553688975

Order = 3
AIC:  -25.95409622765762
BIC:  -24.706005707337322

Order = 4
AIC:  -25.83133397487291
BIC:  -24.190074048568135

Order = 5
AIC:  -25.74575918999568
BIC:  -23.706885036524362

Order = 6
AIC:  -25.61315581085043
BIC:  -23.17213761146995

Order = 7
AIC:  -25.42642291473104
BIC:  -22.578643619895402

Order = 8
AIC:  -25.163320499840506
BIC:  -21.90407352319604

Order = 9
AIC:  -25.045380281935955
BIC:  -21.3698671188398

Order = 10
AIC:  -25.156306808030536
BIC:  -21.059634575771167



In [25]:
data1

,Open,High,Low,Close
0,0.140240,0.072381,0.073518,0.021463
1,0.049331,0.000000,0.064032,0.017488
2,0.096547,0.038730,0.120158,0.060413
3,0.110641,0.028571,0.090909,0.023052
4,0.089500,0.012698,0.079051,0.023052
...,...,...,...,...
140,0.463002,0.408254,0.527273,0.510334
141,0.506695,0.396190,0.538340,0.477742
142,0.477097,0.358730,0.495652,0.446741
143,0.422128,0.588571,0.442688,0.722576


In [26]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0.0,0.35734,1,28
